In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy


In [ ]:
path = glob('./test/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [ ]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./test/Cars143.xml,93,130,196,207
1,./test/licensed_car87.xml,785,1080,651,742
2,./test/licensed_car16.xml,292,489,258,320
3,./test/Cars211.xml,28,387,64,235
4,./test/licensed_car136.xml,155,328,201,274


In [ ]:
def parsing(path):
    parser = xet.parse(path).getroot()
    name = parser.find('filename').text
    filename = f'./test/{name}'
    if filename[-1] != 'g':
        filename += '.jpeg'

    # width and height
    parser_size = parser.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    
    return filename, width, height
df[['filename','width','height']] = df['filepath'].apply(parsing).apply(pd.Series)
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./test/Cars143.xml,93,130,196,207,./test/Cars143.png,400,267,0.278750,0.754682,0.092500,0.041199
1,./test/licensed_car87.xml,785,1080,651,742,./test/licensed_car87.jpeg,1800,1200,0.518056,0.580417,0.163889,0.075833
2,./test/licensed_car16.xml,292,489,258,320,./test/licensed_car16.jpeg,769,417,0.507802,0.693046,0.256177,0.148681
3,./test/Cars211.xml,28,387,64,235,./test/Cars211.png,400,295,0.518750,0.506780,0.897500,0.579661
4,./test/licensed_car136.xml,155,328,201,274,./test/licensed_car136.jpeg,500,332,0.483000,0.715361,0.346000,0.219880


In [ ]:
# center_x, center_y, width , height
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']
df.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./test/Cars143.xml,93,130,196,207,./test/Cars143.png,400,267,0.278750,0.754682,0.092500,0.041199
1,./test/licensed_car87.xml,785,1080,651,742,./test/licensed_car87.jpeg,1800,1200,0.518056,0.580417,0.163889,0.075833
2,./test/licensed_car16.xml,292,489,258,320,./test/licensed_car16.jpeg,769,417,0.507802,0.693046,0.256177,0.148681
3,./test/Cars211.xml,28,387,64,235,./test/Cars211.png,400,295,0.518750,0.506780,0.897500,0.579661
4,./test/licensed_car136.xml,155,328,201,274,./test/licensed_car136.jpeg,500,332,0.483000,0.715361,0.346000,0.219880


In [ ]:
df_train = df.iloc[:500]
df_test = df.iloc[500:]

In [ ]:
train_folder = './yolov5/data_images/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()

test_folder = './yolov5/data_images/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x,y, w, h in values:
    image_name = os.path.split(fname)[-1]
    txt_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_name+'.txt')
    
    # copy each image into the folder
    copy(fname,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {x} {y} {w} {h}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()



In [ ]:

!python3 ./yolov5/train.py --data ./data.yaml --cfg ./yolov5/models/yolov5s.yaml --batch-size 8 --name Model --epochs 30


wandb: Currently logged in as: nachiketh. Use `wandb login --relogin` to force relogin
train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=./data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=Model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/home/nachiketh/MI/YOLO': No such file or directory
YOLOv5 🚀 2022-11-26 Python-3.10.6 torch-1.13.0+cu117 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, war

In [ ]:
!python3 ./yolov5/export.py --weight ./yolov5/runs/train/Model5/weights/best.pt --include torchscript onnx

export: data=yolov5/data/coco128.yaml, weights=['./yolov5/runs/train/Model5/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
fatal: cannot change to '/home/nachiketh/MI/YOLO': No such file or directory
YOLOv5 🚀 2022-11-26 Python-3.10.6 torch-1.13.0+cu117 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from yolov5/runs/train/Model5/weights/best.pt with output shape (1, 25200, 6) (13.7 MB)

TorchScript: starting export with torch 1.13.0+cu117...
TorchScript: export success ✅ 1.7s, saved as yolov5/runs/train/Model5/weights/best.torchscript (27.2 MB)

ONNX: starting export with onnx 1.12.0...
ONNX: export success ✅ 0.8s, saved as yolov5/runs/train